<a href="https://colab.research.google.com/github/altinodantas/rabscraping/blob/main/RABScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import dos recursos necessários**

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

**Classe para tratamento de exceção**

Classe de erro personalizada para lidar com tipos de arquivo suportados para salvamento

In [2]:
class InvalidFileError(Exception):

  def __init__(self, *args):
    if args:
      self.mensagem = args[0]
    else:
      self.mensagem = None

  def __str__(self):
    if self.mensagem:
      return "Tipo de arquivo não suportado :\
              tipo informado: {0}\
              informe .xlsx ou .csv".format(self.mensagem)
    else:
      return "Tipo de arquivo não suportado. Informe xlsx ou csv"

**Classe principal**

In [3]:
class RABScraping:
	"""
    Class usada para realizar scraping na página do RAB a partir de uma lista de matrículas e salvar em arquivo csv ou excel
    ...
    Atributos
    ----------
    __lista_dados : list
        Uma lista com dicionários no formato {'matricula':'XX-UUU', 'Proprietário': 'Altino Dantas'}
	__df : DataFrame
		DataFrame contendo a consolidação do dados de __lista_dados
    Métodos
    -------
    obter_dados()
        Retorna um DataFrame pandas com a lista de aeronaves encontradas
	salvar_arquivo(saida="lista_aeronaves.csv")
        Salva um arquivo com a lista das aeronaves encontradas. O arquivo pode ser .xlsx ou .csv
	
    """

	def __init__(self, matriculas, verbose=False):

		if not isinstance(matriculas, list) and not isinstance(matriculas, tuple):
			raise TypeError(f"O parâmetro matrícula deve ser uma lista")

		self.__lista_dados = []
        
		for m in matriculas:
			m = m.strip()

			if verbose:
				print(f"Processando a matrícula: {m} | início")

			html = requests.get(f"https://sistemas.anac.gov.br/aeronaves/cons_rab_resposta.asp?textMarca={m}").content
			soup = BeautifulSoup(html, 'html.parser')

			tabela = soup.find("table", class_="table table-hover")
			linhas = tabela.find_all('tr')
			dados = {'Matricula' : m}

			for ln in linhas:
				texto = ln.text.strip()
				texto = texto.split('\n')
				campo = texto[0].split(":")
				valor = texto[-1].replace("\t","")     
				dados[campo[0]] = valor

			if(dados['Modelo'] != 'Modelo:'):   
				self.__lista_dados.append(dados) 
			else:
				if verbose:     
					print(f"--- Matrícula {m} não encontrada ---")

			if verbose:
				print(f"Processando a matrícula: {m} | fim")

		self.__df = pd.DataFrame(self.__lista_dados, columns=dados.keys())    
		self.__df.sort_values(by=['Matricula'], inplace=True)
		self.__df = self.__trata_dados(self.__df)

	def obter_dados(self):
	
		return self.__df

	def __trata_dados(self, df):

		aux_df = df
		aux_df['Ano de Fabricação'] = aux_df['Ano de Fabricação'].astype('int32')
		aux_df['Número da Matrícula'] = aux_df['Número da Matrícula'].astype('int32')
		aux_df['Número Máximo de Passageiros'] = aux_df['Número Máximo de Passageiros'].astype('int32')
		
		aux_df[['KG','unidade']] = aux_df['Peso Máximo de Decolagem'].str.split('-', expand=True)
		aux_df['Peso Máximo de Decolagem'] = aux_df['KG'].astype('int32')
		aux_df.drop(['KG','unidade'], axis=1, inplace=True)
		aux_df.rename(columns={'Peso Máximo de Decolagem':'Peso Máximo de Decolagem (KG)'}, inplace=True)

		return aux_df

	def salvar_arquivo(self, saida="lista_aeronaves.csv", dados = pd.DataFrame()):

		if dados.empty:
			dados = self.__df

		tipo_arquivo = saida.split('.')[1]
		if tipo_arquivo == 'csv':
			pd.DataFrame.to_csv(dados, saida, columns=dados.columns, index=False, encoding="utf-8")
		elif tipo_arquivo == 'xlsx':
			pd.DataFrame.to_excel(dados, saida, columns=dados.columns, index=False, encoding="utf-8")
		else:
			raise InvalidFileError(tipo_arquivo)

**Utilização**

Criação de uma instância da classe RABScraping informando as matrículas a serem consultadas

In [7]:
lista_matriculas = ['PR-GUM','PS-AEH', 'PR-PJN', 'XE', 'PR-TOL']

aeronaves = RABScraping(matriculas=lista_matriculas, verbose=True)
df = aeronaves.obter_dados()

Processando a matrícula: PR-GUM | início
Processando a matrícula: PR-GUM | fim
Processando a matrícula: PS-AEH | início
Processando a matrícula: PS-AEH | fim
Processando a matrícula: PR-PJN | início
Processando a matrícula: PR-PJN | fim
Processando a matrícula: XE | início
--- Matrícula XE não encontrada ---
Processando a matrícula: XE | fim
Processando a matrícula: PR-TOL | início
Processando a matrícula: PR-TOL | fim


In [9]:
df.T

,0,2,3,1
Matricula,PR-GUM,PR-PJN,PR-TOL,PS-AEH
Proprietário,"UMB BANK, NATIONAL ASSOCIATION",WILMINGTON TRUST SP SERVICES (DUBLIN) LIMITED,MARCELO SOUZA DUARTE,CRUZEIRO AIRCRAFT FUNDING LLC
CPF/CNPJ,07575651000159,09296295000160,00677645171,09296295000160
Operador,GOL LINHAS AÉREAS S.A,AZUL LINHAS AEREAS BRASILEIRAS S.A,MARCELO SOUZA DUARTE,AZUL LINHAS AEREAS BRASILEIRAS S.A.
Fabricante,BOEING COMPANY,EMBRAER,BEECH AIRCRAFT,YABORA INDUSTRIA AERONAUTICA S.A
Ano de Fabricação,2011,2019,1989,2020
Modelo,737-8EH,ERJ 190-400,58,ERJ 190-400
Número de Série,35846,19020018,TH-1554,19020035
Tipo ICAO,B738,E295,BE58,E295
Tipo de Habilitação para Pilotos,B739,E179,MLTE,E179
